# Affine transformation matrix decomposition

Decomposition of affine transformations into translation, rotation and scales.

Start with a 4x4 homogeneous matrix H of the form

$H = \begin{bmatrix}
 a & b & c & d \\ 
 e & f & g & h \\ 
 i & j & k & l \\ 
 0 & 0 & 0 & 1 \\
\end{bmatrix}$

In [35]:
import numpy as np
np.set_printoptions(precision=3, suppress=True)

H = np.array([(6.15664,3.76798,9.9293,83.8339),
              (-15.4335,3.99544,-6.80062,-23.3666),
              (-24.9755,-1.54013,6.65004,44.6757),
              (0,0,0,1)], dtype=float)
print(H)

[[  6.157   3.768   9.929  83.834]
 [-15.434   3.995  -6.801 -23.367]
 [-24.976  -1.54    6.65   44.676]
 [  0.      0.      0.      1.   ]]


## Translation

Obtain the translation $T$ from the last column and the linear component $L$ from the 3x3 upper-left block.

$H = T L$

In [36]:
T = np.eye(4)
T[:3,3] = H[:3,3]
print(T)

[[  1.      0.      0.     83.834]
 [  0.      1.      0.    -23.367]
 [  0.      0.      1.     44.676]
 [  0.      0.      0.      1.   ]]


In [37]:
L = H.copy()
L[:3,3] = 0
print(L)

[[  6.157   3.768   9.929   0.   ]
 [-15.434   3.995  -6.801   0.   ]
 [-24.976  -1.54    6.65    0.   ]
 [  0.      0.      0.      1.   ]]


Check that $H=TL$:

In [38]:
np.allclose(H, T @ L)

True

## Rotation

Calculate the polar decomposition of $L$ to obtain rotation $R$ and stretch $K$ matrices:

$L = R K$

$H = T R K$

In [39]:
from scipy.linalg import polar

R, K = polar(L)
print(R)

[[ 0.205  0.661  0.722  0.   ]
 [-0.514  0.7   -0.495  0.   ]
 [-0.833 -0.27   0.484  0.   ]
 [ 0.     0.     0.     1.   ]]


In [40]:
print(K)

[[29.998 -0.     0.     0.   ]
 [-0.     5.704  0.     0.   ]
 [ 0.     0.    13.75   0.   ]
 [ 0.     0.     0.     1.   ]]


The determinant of a rotation matrix must be positive. When the determinant of $R$ is negative it is necessary to change the sign of the linear part of both matrices:

In [41]:
if np.linalg.det(R) < 0:
    R[:3,:3] = -R[:3,:3]
    K[:3,:3] = -K[:3,:3]
print(R)

[[ 0.205  0.661  0.722  0.   ]
 [-0.514  0.7   -0.495  0.   ]
 [-0.833 -0.27   0.484  0.   ]
 [ 0.     0.     0.     1.   ]]


In [42]:
print(K)

[[29.998 -0.     0.     0.   ]
 [-0.     5.704  0.     0.   ]
 [ 0.     0.    13.75   0.   ]
 [ 0.     0.     0.     1.   ]]


Check that $L=RK$ and $H=TRK$:

In [28]:
np.allclose(L, R @ K)

True

In [29]:
np.allclose(H, T @ R @ K)

True

## Scale
The stretch matrix $K$ can be further analyzed to obtain up to three scale matrices.

$K = S_1 S_2 S_3$

$H = T R S_1 S_2 S_3$

Each eigenvalue of $K$ represents a scale factor $f_i$, and its corresponding eigenvector $X_i$ represents the scaling axis. Each pair ($f_i, X_i$) can be used to construct a scale matrix:

$S_i = I + X_i X_i^T (f_i-1)$

When $f_i = 1$ the result is an identity matrix, so the pair can be discarded.

In [30]:
f, X = np.linalg.eig(K)
print(f)

[29.998 13.75   5.704  1.   ]


In [31]:
print(X)

[[ 1. -0. -0.  0.]
 [-0.  0. -1.  0.]
 [-0. -1. -0.  0.]
 [ 0.  0.  0.  1.]]


In [32]:
S = []
for factor, axis in zip(f, X.T):
    if not np.isclose(factor, 1):
        scale = np.eye(4) + np.outer(axis, axis) * (factor-1)
        S.append(scale)
        print(scale)

[[29.998 -0.    -0.     0.   ]
 [-0.     1.     0.     0.   ]
 [-0.     0.     1.     0.   ]
 [ 0.     0.     0.     1.   ]]
[[ 1.   -0.    0.    0.  ]
 [-0.    1.   -0.    0.  ]
 [ 0.   -0.   13.75  0.  ]
 [ 0.    0.    0.    1.  ]]
[[1.    0.    0.    0.   ]
 [0.    5.704 0.    0.   ]
 [0.    0.    1.    0.   ]
 [0.    0.    0.    1.   ]]


Check that $K = S_1S_2S_3$ and $H = TRS_1S_2S_3$:

In [33]:
np.allclose(K, S[0] @ S[1] @ S[2])

True

In [34]:
np.allclose(H, T @ R @ S[0] @ S[1] @ S[2])

True

## Inverse Matrix

In [47]:
import math
def transposeMatrix(m):
    return list(map(list,zip(*m)))

def getMatrixMinor(m,i,j):
    return [row[:j] + row[j+1:] for row in (m[:i]+m[i+1:])]

def getMatrixDeternminant(m):
    #base case for 2x2 matrix
    if len(m) == 2:
        return m[0][0]*m[1][1]-m[0][1]*m[1][0]

    determinant = 0
    for c in range(len(m)):
        determinant += ((-1)**c)*m[0][c]*getMatrixDeternminant(getMatrixMinor(m,0,c))
    return determinant

def getMatrixInverse(m):
    determinant = getMatrixDeternminant(m)
    #special case for 2x2 matrix:
    if len(m) == 2:
        return [[m[1][1]/determinant, -1*m[0][1]/determinant],
                [-1*m[1][0]/determinant, m[0][0]/determinant]]

    #find matrix of cofactors
    cofactors = []
    for r in range(len(m)):
        cofactorRow = []
        for c in range(len(m)):
            minor = getMatrixMinor(m,r,c)
            cofactorRow.append(((-1)**(r+c)) * getMatrixDeternminant(minor))
        cofactors.append(cofactorRow)
    cofactors = transposeMatrix(cofactors)
    for r in range(len(cofactors)):
        for c in range(len(cofactors)):
            cofactors[r][c] = cofactors[r][c]/determinant
    return cofactors

D = [[29.9979, 0.0, 0.0, 30.0021], [0, 5.70379, 0.0, 5.7038], [0, 0, 13.75, 13.75], [0,0,0,1]]

D_inv = getMatrixInverse(D)
print(D_inv)

[[0.033335666830011434, -0.0, 0.0, -1.000140009800686], [-0.0, 0.1753220227252406, -0.0, -1.0000017532202274], [0.0, -0.0, 0.07272727272727274, -1.0], [-0.0, 0.0, -0.0, 1.0]]
